# Project: Planning my next holidays ☀️

## on va  nettoyer et analyser les données que nous avons collectées sur les villes et les hôtels. À la fin, il faut les enregistrer dans un seau S3 de notre compte AWS.

## Import librairies

In [1]:
!pip install boto3
# !pip install plotly==4.8.1
# !jupyter labextension install jupyterlab-plotly@4.8.1

     |████████████████████████████████| 132 kB 10.0 MB/s            
  Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
     |████████████████████████████████| 8.6 MB 21.2 MB/s            


In [2]:
import pandas as pd
import glob
import plotly.express as px
import boto3

## 1. Lire le fichier `.csv` qui contient des informations sur les villes et la météo.

In [3]:
cities = pd.read_csv('res/1_destinations.csv')
cities.head()

,city_id,name,latitude,longitude,main_weather,expected_rain,day_temperature,rank,inverted_rank
0,22,Avignon,43.949249,4.805901,Clear,0.000,11.65125,1,33
1,26,Saintes Maries de la mer,43.452277,4.428717,Clear,0.000,11.27375,1,33
2,16,Lyon,45.757814,4.832011,Clear,0.000,8.66500,1,33
3,24,Nimes,43.837425,4.360069,Clouds,0.035,11.66500,4,32
4,25,Aigues Mortes,43.565823,4.191284,Clouds,0.038,11.58250,5,31


## 2. Lire les fichiers `.json` contenant les informations sur les hôtels et les enregistrez dans un seul Dataframe pandas. "Vous pouvez enregistrer autant d'informations que vous le souhaitez, mais n'oubliez pas d'enregistrer au moins ces informations importantes" :

### - le nom de la ville
### - id de la ville (vous le trouverez dans le fichier `.csv` sur les villes)
### - créer une colonne contenant un identifiant unique de l'hôtel (hotel_id)
### - le nom de l'hôtel

## 👀 _Si vous stockez des informations textuelles, assurez-vous de les nettoyer de façon à ce qu'elles soient lisibles._

In [4]:
# Récupérer tous les fichiers du dossier `res` en commençant par `2_hotels_` et 
# en finissant par `.json`

hotel_files = glob.glob('res/2_hotels_*.json')

# Créer un nouveau DataFrame

hotels = pd.DataFrame(columns = ['city_id', 'city_name', 'hotel_id', 'name', 'url', 'latitude', 'longitude', 'score', 'description'])

# Iterate sur tous les fichiers JSON

for f in hotel_files:
    city_name = f.split('_')[2].split('.')[0].replace("-"," ")
    city_id = cities.loc[cities['name'] == city_name,'city_id'].values[0]
    
    print("Processing {}".format(city_name))
    
    # Lire les fichiers json et ajouter hotel_id, city_id et city_name dans les DataFrames
    
    temp_dataset = pd.read_json(f)
    temp_dataset = temp_dataset.reset_index().rename({'index': 'hotel_id'}, axis = 1)
    temp_dataset.loc[:,'city_id'] = city_id
    temp_dataset.loc[:,'city_name'] = city_name
    
   # Nettoyer les champs de texte

    temp_dataset.loc[:, 'name'] = temp_dataset['name'].str.replace('\n', '')
    temp_dataset.loc[:, 'url'] = temp_dataset['url'].str.replace('\n', '')
    temp_dataset.loc[:, 'description'] = temp_dataset['description'].str.replace('\n', '')
    
   # Extraire la latitude et la longitude
    
    temp_dataset.loc[:,'coords'] = temp_dataset['coords'].str.split(',')
    temp_dataset.loc[:, 'longitude'] = temp_dataset['coords'].apply(lambda x : x[0])
    temp_dataset.loc[:, 'latitude'] = temp_dataset['coords'].apply(lambda x : x[1])
    
    # Suppression de la colonne coords (qui n'est plus utile)
    
    temp_dataset = temp_dataset.drop('coords', axis=1)
    
    # Ajouter au cadre de données de l'hôtel
    
    hotels = hotels.append(temp_dataset)

Processing Strasbourg


ValueError: Expected object or value

In [ ]:
# Contrôle
hotels.head()

In [ ]:
# Convert columns to convenient types
hotels.loc[:, 'score'] = hotels['score'].str.replace(',','.')
hotels = hotels.astype({'city_id': int,
                        'hotel_id': int,
                        'latitude': float,
                        'longitude': float,
                        'score': float})

In [ ]:
# Contrôle
hotels.head()

### 3. Enregistrez le DataFrame contenant toutes les informations sur les hôtels dans un fichier `.csv` :

In [ ]:
# Save hotels DataFrame into .csv file
hotels.to_csv('res/3_hotels.csv', index=False)

### 4. Facultatif : utilisez Plotly pour afficher tous les hôtels sur une carte :

In [ ]:
hotels_with_score = hotels.loc[hotels['score'].notnull(),:]

fig = px.scatter_mapbox(hotels_with_score, lat="latitude", lon="longitude", hover_name = 'name', zoom = 4,
                        hover_data = ['description'],
                        color = 'score', color_continuous_scale = 'thermal',
                        mapbox_style="carto-positron")
fig.show('iframe')

### 5. Utilisez `boto3` pour enregistrer les DataFrames sur les villes et les hôtels dans des fichiers `.csv` situés dans un nouveau seau S3 de votre compte AWS :

In [ ]:
# Initiate a new session (if .aws/credentials does not exist)
# session = boto3.Session(aws_access_key_id="YOUR_ACCESS_KEY_ID", 
#                         aws_secret_access_key="YOUR_SECRET_ACCESS_KEY")

# Declare s3 object and create a new bucket
s3 = boto3.resource("s3")  # s3 = session.resource("s3")
bucket_name = s3.create_bucket(Bucket="m03-jedha-project-040121")

# Export hotels to CSV file and upload it
hotels_csv = hotels.to_csv(index=False)
put_object = bucket_name.put_object(Key = "hotels.csv", Body = hotels_csv)

# Do the same for cities
cities_csv = cities.to_csv(index=False)
put_object = bucket_name.put_object(Key = "cities.csv", Body = cities_csv)